In [1]:
using Markdown, LinearAlgebra, Roots, Random, ForwardDiff, NLsolve

In [5]:
# Example 1: one constraint with diagonal matrix
let
	m = 3
	n = 1
	A_vn = [diagm([1,1,0])]
	b_vn = [2]
	err_val = 0.000001
	
	Y_vn = diagm(ones(m))

	f(x) = sum(exp.(x)) #vector function that induces the spectral function
	∇F(X) = log(X)
	∇Fc(X) = exp(X)

	X_opt, γ_opt = @time project_QE(f,∇F,∇Fc,A_vn,b_vn,Y_vn,error=err_val,maxits=10000)

	print("X_opt = ",X_opt,'\n')
	print("λ_opt = ",γ_opt,'\n')

	# Check KKT
	print("feasibility ",[dot(A_vn[i],X_opt) for i in 1:n] - b_vn .<= err_val, ". Want ", [dot(A_vn[i],X_opt) for i in 1:n] - b_vn, " close to 0",'\n')
	print("KKT close to 0: ", ∇F(X_opt) - ∇F(Y_vn) + sum(γ_opt.*A_vn) .<= err_val ,'\n')
	print("eigenvalues greater than 0: ",eigvals(X_opt) .> 0,'\n')

	# Errors
	print("the gamma error is: ", abs(γ_opt[1] - 0),'\n')
	print("the X error is: ", norm(X_opt-diagm(ones(m))),'\n')
end

gamma0 is: [0.3462950925745081;;]
iteration: 5, norm(y1-y0): 3.0830893393840597e-12
 64.664598 seconds (43.79 M allocations: 2.729 GiB, 3.75% gc time, 99.95% compilation time)
X_opt = [1.000000000003083 0.0 0.0; 0.0 1.000000000003083 0.0; 0.0 0.0 1.0]
λ_opt = [-3.082987424379846e-12;;]
feasibility Bool[1]. Want [6.1661786787681194e-12] close to 0
KKT close to 0: Bool[1 1 1; 1 1 1; 1 1 1]
eigenvalues greater than 0: Bool[1, 1, 1]
the gamma error is: 3.082987424379846e-12
the X error is: 4.360146757764844e-12


In [6]:
# Example 2: two contraints with random matrices
let
	m = 3
	n = 2
	# A_vn = [diagm([1,1,0]),diagm([0,1,1])] #must be in form: list of m nxn matrices
	A_vn = [Symmetric(10 .*rand(3,3)),Symmetric(7 .*rand(3,3))]
	b_vn = [2 1] #must be in form: 1xm matrix
	err_val = 0.000001
	
	Y_vn = diagm(ones(m))

	f(x) = sum(exp.(x)) #vector function that induces the spectral function
	∇F(X) = log(X)
	∇Fc(X) = exp(X)

	X_opt, γ_opt = @time project_QE(f,∇F,∇Fc,A_vn,b_vn,Y_vn,error=err_val,maxits=10000)

	print("X_opt = ",X_opt,'\n')
	print("λ_opt = ",γ_opt,'\n')

	# Check KKT
	print("feasibility: ",reshape([dot(A_vn[i],X_opt) for i in 1:size(A_vn)[1]],size(A_vn)[1],1) - b_vn' .<= err_val, ". Want ", reshape([dot(A_vn[i],X_opt) for i in 1:size(A_vn)[1]],size(A_vn)[1],1) - b_vn', " close to 0",'\n')
	
	print("KKT close to 0: ", ∇F(X_opt) - ∇F(Y_vn) + (γ_opt*A_vn)[1] .<= err_val ,'\n')
	
	print("eigenvalues greater than 0: ",eigvals(X_opt) .> 0,'\n')
	
	γ1_theo = -γ_opt[2]-log(1-exp(-γ_opt[2])) #pretty darn close to analytic answer
	γ_theo = [γ1_theo γ_opt[2]]
	
	# Errors
	print("the gamma error is: ", norm(γ_theo-γ_opt),'\n')
	print("the X error is: ", norm(X_opt-diagm([exp(-γ_theo[1]), exp(-(γ_theo[1] +γ_theo[2])), exp(-γ_theo[2])])),'\n')
end

gamma0 is: [0.393571541818456 0.6569000531170608]
iteration: 43, norm(y1-y0): 6.519041792662065e-7
  6.327675 seconds (3.77 M allocations: 273.750 MiB, 1.24% gc time, 99.07% compilation time)
X_opt = [0.11120655443413807 -0.0196819759835805 -0.0025000602864730355; -0.0196819759835805 0.25008732587666865 -0.3583875157158306; -0.0025000602864730355 -0.3583875157158306 0.7060445089616877]
λ_opt = [0.07189973765767474 0.3445262782681904]
feasibility: Bool[0; 0;;]. Want [1.0452266402172228e-5; 4.705491566836528e-6;;] close to 0
KKT close to 0: Bool[1 1 1; 1 1 1; 1 1 1]
eigenvalues greater than 0: Bool[1, 1, 1]
the gamma error is: 0.8164811532730502
the X error is: 0.5911480053947579


In [11]:
# Example 3: 
let
	m = 4
	n = 6
	# A_vn = [diagm([1,1,0]),diagm([0,1,1])] #must be in form: list of m nxn matrices
	A_vn = [Symmetric(10 .*rand(n,n)),Symmetric(7 .*rand(n,n)),Symmetric(4 .*rand(n,n)),Symmetric(8 .*rand(n,n))]
	b_vn = [2 1 3 5] #must be in form: 1xm matrix
	err_val = 0.000001
	
	Y_vn = diagm(ones(n))

	f(x) = sum(exp.(x)) #vector function that induces the spectral function
	∇F(X) = log(X)
	∇Fc(X) = exp(X)

	X_opt, γ_opt = @time project_QE(f,∇F,∇Fc,A_vn,b_vn,Y_vn,error=err_val,maxits=10000)

	print("X_opt = ",X_opt,'\n')
	print("λ_opt = ",γ_opt,'\n')

	# Check KKT
	print("feasibility: ",reshape([dot(A_vn[i],X_opt) for i in 1:size(A_vn)[1]],size(A_vn)[1],1) - b_vn' .<= err_val, ". Want ", reshape([dot(A_vn[i],X_opt) for i in 1:size(A_vn)[1]],size(A_vn)[1],1) - b_vn', " close to 0",'\n')
	
	print("KKT close to 0: ", ∇F(X_opt) - ∇F(Y_vn) + (γ_opt*A_vn)[1] .<= err_val ,'\n')
	
	print("eigenvalues greater than 0: ",eigvals(X_opt) .> 0,'\n')
end

gamma0 is: [0.034954716470702474 -0.1013013323302212 0.8222494099155575 0.09185436248200252]
iteration: 7, norm(y1-y0): 5.822909748089552e-8
  1.217354 seconds (477.31 k allocations: 38.006 MiB, 97.46% compilation time)
X_opt = [1.0004619351895343 -0.19491893288797332 -0.3291500790701174 -0.10160239978460069 -0.34656167628147233 -0.09265501406660769; -0.19491893288797332 0.5358903850051054 -0.13875452678324782 -0.055861666665844745 0.09073230766953318 -0.06287676512707863; -0.3291500790701174 -0.13875452678324782 0.9088803874504522 0.03882186714488447 -0.16751181956687372 -0.25717989356688803; -0.10160239978460069 -0.055861666665844745 0.03882186714488447 0.4603749279170144 -0.1175212294995559 0.058098051782855556; -0.34656167628147233 0.09073230766953318 -0.16751181956687372 -0.1175212294995559 0.8194859980013043 -0.2445226105378603; -0.09265501406660769 -0.06287676512707863 -0.25717989356688803 0.058098051782855556 -0.2445226105378603 0.6275448272663195]
λ_opt = [-0.03278298514119997

In [2]:
# Newton's method
function newton2(h, Dh, yo, tol;maxiter=1000)
	iter = 0
	y1 = ones(size(yo))
	d0 = norm(h(yo)).*ones(size(yo)) #copy(yo)
	for i in 1:maxiter
		iter += 1

		dfunc(x) = Dh(yo,x) - h(yo)
		dsol = nlsolve(dfunc,d0)
		d1= dsol.zero
		y1 = yo - 1*d1

		if mod(iter,50)==0
			print("iteration: ", iter,", norm(y1-y0): ",norm(y1-yo),'\n')
			# print("α is: ",α,'\n')
		end
			
		if (norm(y1-yo)) < tol 
			print("iteration: ", iter,", norm(y1-y0): ",norm(y1-yo),'\n')
			break
		elseif isnan(norm(y1-yo)) 
			print("norm undefined",'\n')
			print("nabla h is: ",Dh(yo,d0),'\n')
			print("h is: ",h(yo),'\n')
			break
		end
		
		yo = copy(y1)
		d0 = copy(d1)
	end
	if iter==maxiter
		print("maximum iterations reached. program terminated",'\n')
	end
	return yo
end

newton2 (generic function with 1 method)

In [3]:
# matrix function derivative
function mat_der(f,X,H)
	E = eigen(X)
	Λ = E.values # vector of eigenvalues
	W = E.vectors # matrix of eigenvectors

	onediv = zeros(size(X))
	for i=1:length(Λ)
		for j=1:length(Λ)
			if Λ[i] == Λ[j]
				onediv[i,j] = ForwardDiff.derivative(f,Λ[i])
			else
				onediv[i,j] = (f(Λ[i])-f(Λ[j]))/(Λ[i]-Λ[j])
			end
		end
	end

	return W*(onediv.*(W'*H*W))*W'
end

mat_der (generic function with 1 method)

In [4]:
# quantum entropy projection
function project_QE(f,∇F,∇Fc,A_vn,b_vn,Y_vn;error=0.005,maxits=1000)
	cal_A(X) = reshape([dot(A_vn[i],X) for i in 1:size(A_vn)[1]],size(A_vn)[1],1)
	cal_Ac(x) = (x*A_vn)[1]
	φ(x) = ∇F(Y_vn) - cal_Ac(x)
	D∇Fc(X,H) = mat_der(f,X,H)

	H(γ) = cal_A(∇Fc(φ(γ))) - transpose(b_vn)
	DH(γ,x) = cal_A(D∇Fc(φ(γ),-cal_Ac(x)))

	γ0 = 2*rand(1,size(A_vn)[1]).-1 # range close to that found from bracketing method used in out of box solver
	
	print("gamma0 is: ", γ0,'\n')
	
	γ_opt = newton2(H, DH, γ0, error,maxiter=maxits)
	X_opt = ∇Fc(φ(γ_opt))
	
	# return DH(γ0,γ0),1
	return X_opt, γ_opt
end

project_QE (generic function with 1 method)